In [4]:
# Pima CNN w/ Manual Layer
# Brett Huffman
# Homework 2 - CSCI 5390
# Feb 6, 2022

# Important libraries I'll be using
import numpy as np
import pandas as pd

# Plot libraries
import matplotlib.pyplot as plt             #visualisation
import seaborn as sns                       #visualisation

%matplotlib inline


In [ ]:
# Get the data

!wget https://badriadhikari.github.io/DL/data/pima-diabetes.csv

In [2]:
# Check it out
! head pima-diabetes.csv

Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
6,148,72,35,0,33.6,0.627,50,1
1,85,66,29,0,26.6,0.351,31,0
8,183,64,0,0,23.3,0.672,32,1
1,89,66,23,94,28.1,0.167,21,0
0,137,40,35,168,43.1,2.288,33,1
5,116,74,0,0,25.6,0.201,30,0
3,78,50,32,88,31,0.248,26,1
10,115,0,0,0,35.3,0.134,29,0
2,197,70,45,543,30.5,0.158,53,1


In [9]:
# Convert data into numpy array


# Loading the Data into the Data Frame
df = pd.read_csv('pima-diabetes.csv',error_bad_lines=False,warn_bad_lines=True)

# To display the top 5 rows
df.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [13]:
# A pretty slick normalization function for Pandas
# Based on https://stats.stackexchange.com/questions/70801/how-to-normalize-data-to-0-1-range
def normalizeMaxMin(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        
        print(feature_name, max_value, min_value)

        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [14]:
df = normalizeMaxMin(df)

Pregnancies 17 0
Glucose 199 0
BloodPressure 122 0
SkinThickness 99 0
Insulin 846 0
BMI 67.1 0.0
DiabetesPedigreeFunction 2.42 0.078
Age 81 21
Outcome 1 0


In [15]:
# To display the top 5 rows
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333,1.0
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667,0.0
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333,1.0
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000,0.0
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000,1.0


In [16]:
# Randomize before splitting
from sklearn.utils import shuffle
df = shuffle(df)

In [17]:
# Split the dataset for the actual analysis
# into Training and Validation data sets
npdata = np.asarray(df)
index_30percent = int(0.3 * len(npdata[:, 0]))
print("Training split: ", index_30percent)

# Split the dataset and carve off last column.  This column
# is our result set
XVALID = npdata[:index_30percent, :-1]
YVALID = npdata[:index_30percent, -1]
XTRAIN = npdata[index_30percent:, :-1]
YTRAIN = npdata[index_30percent:, -1]

Training split:  230


In [21]:
print(XTRAIN.shape)
print(YTRAIN.shape)
print(XVALID.shape)
print(YVALID.shape)

(538, 8)
(538,)
(230, 8)
(230,)


In [22]:
# Prepare to generate the model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, AlphaDropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [25]:
from keras import activations
# Custom Layer Code
# A lot of this is from the textbook - Deep Learning with Python 2nd Ed., Listing 3.22
class SimpleDense(keras.layers.Layer):
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation

  def build(self, input_shape):
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape=(input_dim, self.units), initializer = "random_normal")
    self.b = self.add_weight(shape=(self.units,), initializer = "zeros")

  def call(self, inputs):
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y

In [34]:
from keras.layers.advanced_activations import ReLU
# Creating a Sequential Model and adding the layers

epochs = 200
batchSize = 20

model = Sequential()

model.add(Dense(8, input_dim=8, activation='relu'))
model.add(SimpleDense(4, keras.activations.relu))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [35]:
model.fit(XTRAIN, YTRAIN, epochs=epochs, batch_size=batchSize, validation_data=(XVALID, YVALID))

Epoch 1/200
27/27 [==============================] - 1s 9ms/step - loss: 0.2430 - accuracy: 0.6580 - val_loss: 0.2419 - val_accuracy: 0.6348
Epoch 2/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2379 - accuracy: 0.6580 - val_loss: 0.2378 - val_accuracy: 0.6348
Epoch 3/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2336 - accuracy: 0.6580 - val_loss: 0.2356 - val_accuracy: 0.6348
Epoch 4/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2311 - accuracy: 0.6580 - val_loss: 0.2341 - val_accuracy: 0.6348
Epoch 5/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2290 - accuracy: 0.6580 - val_loss: 0.2329 - val_accuracy: 0.6348
Epoch 6/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2273 - accuracy: 0.6580 - val_loss: 0.2314 - val_accuracy: 0.6348
Epoch 7/200
27/27 [==============================] - 0s 3ms/step - loss: 0.2253 - accuracy: 0.6580 - val_loss: 0.2300 - val_accuracy: 0.6348
Epoch 8/200
2